<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

## Step 1.1: Configure the environment

In [2]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
# from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_surf, mask_elevation, all_masks, glcm_features, do_prediction
import matplotlib.pyplot as plt
from odc.algo import binary_dilation, mask_cleanup
from skimage.morphology import disk
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [3]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [4]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "serua_revise"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-serua_revise-19082025


In [5]:
gdfs = []
postcards_path = "training-data/"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [6]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

serua_revise_postcard.geojson
serua_postcard.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [7]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [8]:
# filename = "nm-efate-27072025_postcard.geojson"

In [9]:
# training = gpd.read_file(f"training-data/{site}_postcard.geojson")
# training = training.to_crs("EPSG:4326")
# min_lon, min_lat, max_lon, max_lat = training.total_bounds

# bbox = [min_lon, min_lat, max_lon, max_lat]

In [73]:
# # Serua-Deuba
# bbox = [177.85649, -18.30487, 178.01041, -18.25023]

# Coral coast
bbox = [177.50076, -18.23336, 177.85369, -18.18450]

# # North Viti Levu
# bbox = [177.4755, -17.5305, 178.3384, -17.1883]

# # Ba Estuary
# bbox = [177.4755, -17.5305,  177.71596, -17.31072,]

# # Suva
# bbox = [178.33211, -18.19602, 178.55566, -18.09947]

# # South Efate
# bbox = [168.24347, -17.81064, 168.39286, -17.72472]

In [74]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [75]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 81MB
Dimensions:      (y: 570, x: 3929, time: 1)
Coordinates:
  * y            (y) float64 5kB -2.046e+06 -2.046e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 31kB 3.061e+06 3.061e+06 ... 3.101e+06 3.101e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 4MB dask.array<chunksize=(1, 570, 2048), meta=np.ndarray>
    red          (time, y, x) uint16 4MB dask.array<chunksize=(1, 570, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 4MB dask.array<chunksize=(1, 570, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 4MB dask.array<chunksize=(1, 570, 2048), meta=np.ndarray>
    emad         (time, y, x) float32 9MB dask.array<chunksize=(1, 570, 2048), meta=np.ndarray>
    smad         (time, y, x) float32 9MB dask.array<chunksize=(1, 570, 2048), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 4MB dask.array<ch

In [76]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43717 instead
  warnings.warn(


In [77]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [78]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [79]:
scaled

<xarray.Dataset> Size: 242MB
Dimensions:      (y: 570, x: 3929)
Coordinates:
  * y            (y) float64 5kB -2.046e+06 -2.046e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 31kB 3.061e+06 3.061e+06 ... 3.101e+06 3.101e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 18MB 0.0413 0.0421 0.0425 ... 0.3464 0.3869
    red          (y, x) float64 18MB 0.0461 0.0465 0.047 ... 0.0314 0.029 0.0291
    blue         (y, x) float64 18MB 0.0639 0.0645 0.0662 ... 0.0316 0.0338
    green        (y, x) float64 18MB 0.0564 0.0571 0.0576 ... 0.0503 0.0528
    emad         (y, x) float32 9MB 0.1157 0.1157 0.1164 ... 0.1023 0.09469
    smad         (y, x) float32 9MB 2.198e-06 2.316e-06 ... 1.571e-07 1.185e-07
    ...           ...
    swir16       (y, x) float64 18MB 0.0392 0.0395 0.0397 ... 0.1631 0.1628
    swir22       (y, x) float64 18MB 0.0366 0.0368 0.0376 ... 0.0743 0.0738
    coastal      (y, x) float64 18MB 0.0664 0.0666 0.0664 ... 0.0275 0.0272
    rededge1     (y, x) float64 18MB 0.0469 0.047 0.0477 ... 0.0823 0.0818
    rededge2     (y, x) float64 18MB 0.0435 0.0438 0.0442 ... 0.2954 0.2952
    rededge3     (y, x) float64 18MB 0.0439 0.0443 0.0443 ... 0.3751 0.3749

In [80]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [81]:
# WINDOW_SIZE = 9
# LEVELS = 32

# # Input
# max = scaled.blue.max().values
# min = scaled.blue.min().values
# # Scale to 0-LEVELS for GLCM
# img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# # Extract overlapping windows
# patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# # Shape: (rows, cols, win_y, win_x)


In [82]:
# import numpy as np # Ensure numpy is imported if not already

# # Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# # To get the first patch (at y=0, x=0), you would index it like this:
# sample_patch_data = patches[0, 0, :, :]

# # Verify the shape of the extracted sample patch data
# print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# # Call glcm_features directly on this 2D sample data
# sample_result = glcm_features(sample_patch_data)

# # Print the shape of the result to get the number of features
# print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

In [83]:
# # Use apply_ufunc to vectorize over (row, col) dimensions
# result = xr.apply_ufunc(
#     glcm_features,
#     xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
#     input_core_dims=[["win_y", "win_x"]],
#     output_core_dims=[["feature"]],
#     vectorize=True,
#     dask="parallelized",
#     output_dtypes=[np.float32]
# )

# # Add coordinates & names
# pad = WINDOW_SIZE - 1
# result = result.assign_coords({
#     "y": scaled.y[: -pad],
#     "x": scaled.x[: -pad],
#     "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
# })

# result_bands = result.to_dataset(dim="feature")

# # Combine with original
# combined = scaled.copy()
# combined = combined.assign(result_bands)

# combined

In [ ]:
combined = scaled
combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [ ]:
# masked_combined, mask = all_masks(combined, return_mask = True)
# mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [ ]:
# masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site, tiles=basemaps.Esri.WorldImagery)

In [ ]:
swir_mask = (combined.swir16 < 0.06)

In [ ]:
# swir_mask = swir_mask.astype('uint8')
# swir_mask.odc.write_cog("swir_mask.tif", overwrite=True)

In [ ]:
swir_mask = (combined.swir16 > 0.08)

land_threshold = -0.2
land_mask = (combined.mndwi<land_threshold).squeeze()

swir_mask = swir_mask.astype(bool)
land_mask = land_mask.astype(bool)

land_mask = land_mask + swir_mask
# land_mask.plot()

In [ ]:
# land_mask = ~water_mask
land_mask = land_mask.chunk({'x': 512, 'y': 512})

land_mask = land_mask.astype(bool)
land_dilation_radius = 2


dilated_land_mask = binary_dilation(land_mask, radius=land_dilation_radius)

fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Plot Initial Mask
land_mask.plot.imshow(
    ax=axes[0],
    cmap='gray', # 'gray' or 'Greys_r' are good for binary masks
    add_colorbar=False
)
axes[0].set_title(f'Initial Land Mask (NIR > {land_threshold})')
axes[0].set_aspect('equal', adjustable='box')


# Plot Dilated Mask
dilated_land_mask.plot.imshow(
    ax=axes[1],
    cmap='gray', # Use the same colormap for comparison
    add_colorbar=False,
)
axes[1].set_title(f'Dilated Land Mask (Radius={land_dilation_radius})')
axes[1].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

In [ ]:

dilated_land_mask = dilated_land_mask.astype(bool)
# land_mask = land_mask.chunk({'x': 512, 'y': 512})

# land_mask = land_mask.astype(bool)
water_mask = ~dilated_land_mask
# water_mask = ~land_mask
water_mask.plot()


In [ ]:

# # dilated_land_mask = dilated_land_mask.astype(bool)
# land_mask = land_mask.chunk({'x': 512, 'y': 512})

# land_mask = land_mask.astype(bool)
# # water_mask = ~dilated_land_mask
# water_mask = ~land_mask
# water_mask.plot()


In [ ]:

# from scipy.ndimage import binary_dilation
surf_threshold = 0.08

surf_mask = combined.nir > surf_threshold
# surf_mask = combined.nir.where(combined.nir<surf_threshold).squeeze()
# surf_mask = ~surf_mask
# surf_mask = combined.nir.where(surf_threshold)
surf_mask = surf_mask.astype(bool)

surf_mask.plot()

In [ ]:
surf_mask = surf_mask & water_mask
surf_mask.plot()
surf_mask = surf_mask.astype('uint8')
surf_mask.odc.write_cog("surf_combined.tif", overwrite=True)

In [ ]:
# # In mask_surf, after initial_mask_for_dilation is defined:
# # To remove small false positives before dilation

# initial_mask_for_dilation = initial_mask_for_dilation.astype(bool)
# initial_mask_for_dilation = mask_cleanup(initial_mask_for_dilation, [["erosion", 2], ["dilation", 2]])
# # Then apply your main dilation_radius expansion:
# expanded_mask = binary_dilation(initial_mask_for_dilation, radius=20)
# expanded_mask.plot()

In [ ]:
# expanded_mask = expanded_mask.astype('uint8')
# expanded_mask.odc.write_cog("expanded_combined.tif", overwrite=True)

In [ ]:
# In mask_surf function, within the refinement block:
# Define specific thresholds for each band (tune these values based on your imagery)
surf_blue_threshold = 0.27 # Example: Very high Blue reflectance
surf_green_threshold = 0.22 # Example: Very high Green reflectance
surf_red_threshold = 0.15   # Example: Very high Red reflectance
surf_nir_threshold = 0.08   # Example: Very high NIR reflectance

# The initial raw surf mask should now check multiple bands
initial_surf_mask_raw = (combined.blue > surf_blue_threshold) & \
                        (combined.green > surf_green_threshold) & \
                        (combined.red > surf_red_threshold) & \
                        (combined.nir > surf_nir_threshold)
# Then, this `initial_surf_mask_raw` is ANDed with `water_area_mask` as before.
initial_surf_mask_raw = initial_surf_mask_raw & water_mask
# initial_mask_for_dilation = mask_cleanup(surf_mask, [["erosion", 2], ["dilation", 2]])

# initial_surf_mask_raw.plot()


test_surf = initial_surf_mask_raw.astype('uint8')
# test_surf.odc.write_cog("test_surf.tif", overwrite=True)

In [ ]:
test_surf.odc.explore(tiles=basemaps.Esri.WorldImagery)

In [ ]:
initial_mask_for_dilation = test_surf.astype(bool)
# initial_mask_for_dilation = mask_cleanup(initial_mask_for_dilation, [["erosion", 2], ["dilation", 2]])
# Then apply your main dilation_radius expansion:
expanded_mask = binary_dilation(initial_mask_for_dilation, radius=20)
expanded_mask.plot()

In [ ]:
expanded_mask.odc.explore(tiles=basemaps.Esri.WorldImagery)

In [37]:

# surf_mask = expanded_mask.chunk({'x': 512, 'y': 512})

# surf_mask = surf_mask.astype(bool)
# surf_dilation_radius = 20


# dilated_surf_mask = binary_dilation(surf_mask, radius=surf_dilation_radius)

# fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# # Plot Initial Mask
# surf_mask.plot.imshow(
#     ax=axes[0],
#     cmap='gray', # 'gray' or 'Greys_r' are good for binary masks
#     add_colorbar=False
# )
# axes[0].set_title(f'Initial Surf Mask (NIR > {surf_threshold})')
# axes[0].set_aspect('equal', adjustable='box')


# # Plot Dilated Mask
# dilated_surf_mask.plot.imshow(
#     ax=axes[1],
#     cmap='gray', # Use the same colormap for comparison
#     add_colorbar=False
# )
# axes[1].set_title(f'Dilated Surf Mask (Radius={surf_dilation_radius})')
# axes[1].set_aspect('equal', adjustable='box')


# plt.tight_layout()
# plt.show()


In [38]:
dilated_surf_mask = expanded_mask
dilated_surf_mask = dilated_surf_mask.astype('uint8')
dilated_surf_mask.odc.write_cog("masked_combined.tif", overwrite=True)

PosixPath('masked_combined.tif')

In [39]:
# masked_combined = masked_combined.where(~dilated_surf_mask)
# masked_combined.red.plot()

In [40]:
# masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [41]:
# masked_combined = masked_combined.astype('uint8')
# masked_combined.red.odc.write_cog("masked_combined.tif", overwrite=True)

In [42]:
# masked_combined = masked_combined.where(

### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [43]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

NameError: name 'training' is not defined

In [ ]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

In [ ]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

In [ ]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [ ]:
training_array=training_array[standard_schema]

In [ ]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [ ]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [ ]:
training_array["cc_id"].dtype